# Vorbereitung MicroPython

## Installation der Firmware für das Node MCU Microcontroller-Board (ESP 8266)

Herunterladen der Firmware (neuestes Release für einem Flash-Speicher größer 2 MB esp8266-20220117-v1.18.bin) von https://micropython.org/download/esp8266/

Flashen der Firmware auf das Node MCU Mikroprozessor Board, siehe z.B. https://randomnerdtutorials.com/flashing-micropython-firmware-esptool-py-esp32-esp8266/

## Installation des Jupyter Kernels für MicroPython

pip install jupyter_micropython_kernel

python -m jupyter_micropython_kernel.install

## Verbinden mit dem Node MCU über die serielle Schnittstelle

Windows (z.B. über COM6)

In [ ]:
%serialconnect to --port=COM7 --baud=115200

MacOs (z.B. über /dev/ttyUSB0

In [ ]:
%serialconnect to --port=/dev/ttyUSB0 --baud=115200

Test der Kommunikation mit dem Node MCU

In [ ]:
print('Hello World')

# Erster Test mit dem Node MCU Microcontroller

Pin-Belegung des Node MCU:

<img src="./img/NodeMCU8266.png" width="600">

Ein- und Auschalten der eingebauten LED (die LED ist intern auf der Plus-Seite mit dem 3.3 V-Pin und auf der Minus-Seite mit dem GPIO2-Pin verbunden)

In [ ]:
import machine
# Definition von Pin(2) als Ausgang, Pin(2) entspricht gpio2
led = machine.Pin(2, machine.Pin.OUT)

In [ ]:
led.value(0) # Einschalten der LED
# alternativ led.on()

In [ ]:
led.value(1) # Ausschalten der LED
# alternativ led.off()

In [ ]:
# Ein- und Auschalten der LED
import time
for i in range(10):
    led.on()
    time.sleep(0.5)
    led.off()
    time.sleep(0.5)

## Erster Test des Temperatur- und Feuchtetsensors DHT22

Pin-Belegung des DHT22

<img src="./img/DHT22-Sensor.jpg" width="300">

In [ ]:
from machine import Pin
from time import sleep
import dht 

sensor = dht.DHT22(Pin(4))

while True:
  try:
    sleep(2)
    sensor.measure()
    temp = sensor.temperature()
    hum = sensor.humidity()
    print('Temperature: %3.1f C' %temp)
    print('Humidity: %3.1f %%' %hum)
  except OSError as e:
    print('Failed to read sensor.')

## Berechnung der Taupunkttemperatur

Zu einem beliebigen Zustand der Luft (z.B. roter Punkt) gehört ein Taupunkt bei gleichem Wasserdampf-Partialdruck (blauer Punkt).

<img src="./img/Taupunkt1.png" width="500">

Bildquelle: https://de.wikipedia.org/wiki/Taupunkt

Mit diesem Diagramm lässt sich für eine bestimmte Luftemperatur und relativer Feuchte der zugehörige Taupunkt bestimmen:

<img src="./img/Taupunkt2.png" width="600">

Bildquelle: https://de.wikipedia.org/wiki/Taupunkt

Berechnungsformel für die Taupunktemperatur (Quelle: https://de.wikipedia.org/wiki/Taupunkt)

$$ K_1 = 17,62$$ 

$$ K_2 = 243,12 °C$$

$$\tau(\phi,\Theta) = K_2 \cdot \frac{\frac{K_1 \cdot \theta}{K_2 + \theta}+\ln \phi}{\frac{K_1 \cdot K_2}{K_2 + \theta} - \ln \phi}$$

Diese Näherungsformel ist gültig für den Sättigungsdampfdruck über Wasser in einem Bereich von -45 °C bis 60 °C.

1. Umsetzung in eine Python-Funktion

In [ ]:
import math

def theta_dewpoint(phi,theta):
    K1 = 17.62
    K2 = 243.12
    return K2*(K1*theta/(K2+theta)+math.log(phi))/(K1*K2/(K2+theta)-math.log(phi))

2. Anwenden dieser Funktion zur zusätzlichen Ausgabe der Taupunktemperatur

In [ ]:
from machine import Pin
from time import sleep
import dht 

sensor = dht.DHT22(Pin(4))

while True:
  try:
    sleep(2)
    sensor.measure()
    temp = sensor.temperature()
    hum = sensor.humidity()
    tempDp = theta_dewpoint(hum/100,temp)
    print('Temperature: %3.1f °C' %temp)
    print('Dew point temperature: %3.1f °C' %tempDp)
    print('Humidity: %3.1f %%' %hum)
  except OSError as e:
    print('Failed to read sensor.')

## Ergänzendes Tutorial

https://micropython-on-esp8266-workshop.readthedocs.io/en/latest/index.html